In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, cnn_clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
lyrics, max_len, word_list = cnn_clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
words = []
words.append('<PAD>')
for word in word_list:
    words.append(word)

In [8]:
train_lyrics = np.zeros(shape=(len(train_df), max_len))

row_index = 0
for row in train_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        train_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [9]:
print(train_lyrics.shape)

(10098, 1525)


In [10]:
train_genres = one_hot_genres(train_df['Genre'])

In [11]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [12]:
val_genres = one_hot_genres(val_df['Genre'])

In [13]:
val_lyrics = np.zeros(shape=(len(val_df), max_len))

row_index = 0
for row in val_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list and col_index < max_len:
            val_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [14]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [15]:
test_genres = one_hot_genres(test_df['Genre'])

In [16]:
test_lyrics = np.zeros(shape=(len(test_df), max_len))

row_index = 0
for row in test_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list and col_index < max_len:
            test_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [17]:
print(train_lyrics.shape)

(10098, 1525)


In [18]:
# Source: https://github.com/ibrahimcelenli/cnn-word2vec-tweets-classification/blob/master/cnn_word2vec_tweets_classification.ipynb

word2vec = {}
with open('w2v_sg_model.word2vec', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [19]:
# Source: https://github.com/ibrahimcelenli/cnn-word2vec-tweets-classification/blob/master/cnn_word2vec_tweets_classification.ipynb

EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(words), EMBEDDING_DIM))
index = 0
for word in words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
    index+=1


In [20]:
print(embedding_matrix.shape)

(34261, 100)


In [21]:
from keras.layers import Embedding

embedding_layer = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=True)

Using TensorFlow backend.


In [22]:
# For generating loss curve *Add callback function to model fitting*
%load_ext tensorboard

In [78]:
import tensorflow as tf
import datetime
!rm -rf ./logs/ 

In [79]:
log_dir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [80]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6008 (pid 21677), started 4 days, 14:34:39 ago. (Use '!kill 21677' to kill it.)

In [85]:
from keras.models import Model
from keras.layers import Dense, Input, Activation, Conv1D, MaxPooling1D, Dropout, Flatten
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model

sequence_input = Input(shape=(max_len,), dtype='int32')
embedding = embedding_layer(sequence_input)

# # channel 1
# conv1 = Conv1D(128, 2, activation='relu')(embedding)
# drop1 = Dropout(0.2)(conv1)
# pool1 = MaxPooling1D(2)(drop1)
# dense1 = Dense(32, activation='relu')(pool1)
# flat1 = Flatten()(dense1)

# # channel 2
# sequence_input2 = Input(shape=(max_len,), dtype='int32')
# embedded2 = embedding_layer(sequence_input2)
# conv2 = Conv1D(128, 3, activation='relu')(embedded2)
# drop2 = Dropout(0.4)(conv2)
# pool2 = MaxPooling1D(3)(drop2)
# dense2 = Dense(32, activation='relu')(pool2)
# flat2 = Flatten()(dense2)

# # channel 3
# sequence_input3 = Input(shape=(max_len,), dtype='int32')
# embedded3 = embedding_layer(sequence_input3)
# conv3 = Conv1D(128, 4, activation='relu')(embedded3)
# drop3 = Dropout(0.4)(conv3)
# pool3 = MaxPooling1D(4)(drop3)
# dense3 = Dense(32, activation='relu')(pool3)
# flat3 = Flatten()(dense3)

# # channel 4
# sequence_input4 = Input(shape=(max_len,), dtype='int32')
# embedded4 = embedding_layer(sequence_input4)
# conv4 = Conv1D(128, 5, activation='relu')(embedded4)
# drop4 = Dropout(0.4)(conv4)
# pool4 = MaxPooling1D(5)(drop4)
# dense4 = Dense(32, activation='relu')(pool4)
# flat4 = Flatten()(dense4)

# # merge
# merged = concatenate([flat1, flat2, flat3, flat4], axis=-1)
# # interpretation
# dense5 = Dense(32, activation='relu')(merged)
# drop5 = Dropout(0.3)(dense5)
# outputs = Dense(8, activation='softmax')(drop5)


# model = Model(inputs=[sequence_input, sequence_input2, sequence_input3, sequence_input4], outputs=outputs)

conv1 = Conv1D(32, 2, activation='relu')(embedding)
drop1 = Dropout(0.9)(conv1)
pool1 = MaxPooling1D(2)(drop1)
flatten = Flatten()(pool1)
final = Dense(8, activation='softmax')(flatten)
model = Model(sequence_input, final)

plot_model(model, show_shapes=True, to_file='cnn.png')
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 1525)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1525, 100)         3426100   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1524, 32)          6432      
_________________________________________________________________
dropout_13 (Dropout)         (None, 1524, 32)          0         
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 762, 32)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 24384)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 195

In [86]:
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

model.fit(train_lyrics, train_genres, epochs=5, 
          shuffle=True, batch_size=16, validation_data=(val_lyrics, val_genres),
          callbacks=[tensorboard_callback])

Train on 10098 samples, validate on 2142 samples
Epoch 1/5
10098/10098 [==============================] - 27s 3ms/step - loss: 1.3441 - acc: 0.4990 - val_loss: 1.2565 - val_acc: 0.5518
Epoch 2/5
10098/10098 [==============================] - 27s 3ms/step - loss: 0.8712 - acc: 0.6878 - val_loss: 1.2417 - val_acc: 0.5696
Epoch 3/5
10098/10098 [==============================] - 27s 3ms/step - loss: 0.7473 - acc: 0.7365 - val_loss: 1.2425 - val_acc: 0.5728
Epoch 4/5
10098/10098 [==============================] - 27s 3ms/step - loss: 0.6936 - acc: 0.7655 - val_loss: 1.2958 - val_acc: 0.5770
Epoch 5/5
10098/10098 [==============================] - 26s 3ms/step - loss: 0.6542 - acc: 0.7757 - val_loss: 1.3414 - val_acc: 0.5658


In [87]:
# model.fit(val_lyrics, val_genres, epochs=1, 
#           shuffle=True, batch_size=16)

In [84]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)
print("Accuracy: ", results[1])

2160/2160 [==============================] - 1s 349us/step
Accuracy:  0.43287035822868347


In [56]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values